In [1]:
import os
import sagemaker
from sagemaker.tensorflow.serving import Model
import tensorflow as tf
import numpy as np
from sagemaker import get_execution_role
import numpy as np
import cv2
from matplotlib import pyplot as mlt
import boto3
import json
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
my_training_artifact = 's3://sagemaker-us-east-1-452432741922/tf-keras-clasif-2020-05-21-17-57-39-715/output/model.tar.gz'

In [3]:
%%bash -s "$my_training_artifact" 

aws s3 cp $1 ./

download: s3://sagemaker-us-east-1-452432741922/tf-keras-clasif-2020-05-21-17-57-39-715/output/model.tar.gz to ./model.tar.gz


In [4]:
! tar -xvf model.tar.gz 

1/
1/saved_model.pb
1/assets/
1/variables/
1/variables/variables.data-00001-of-00002
1/variables/variables.index
1/variables/variables.data-00000-of-00002


In [5]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()
bucket = sagemaker_session.default_bucket()
prefix = 'tf-serving'

In [6]:
model_data = sagemaker_session.upload_data('model.tar.gz',
                                           bucket,
                                           os.path.join(prefix, 'model'))

In [7]:
model_data

's3://sagemaker-us-east-1-452432741922/tf-serving/model/model.tar.gz'

In [8]:
# The "Model" object doesn't create a SageMaker Model until a Transform Job or Endpoint is created.
tensorflow_serving_model = Model(model_data=model_data,
                                 role=role,
                                 framework_version='2.0',
                                 sagemaker_session=sagemaker_session)

In [9]:
predictor = tensorflow_serving_model.deploy(initial_instance_count=1,
                                            instance_type='ml.m5.xlarge')

-------------!

In [10]:
train_data_gen_args = dict(rescale=1./255)

data_gen_args = dict(target_size=(224, 224),
        batch_size=16,
        shuffle=True,
        #color_mode='grayscale',
        class_mode='categorical')

In [11]:
gen = ImageDataGenerator(**train_data_gen_args)

In [12]:
test_set = gen.flow_from_directory('../Classificacion-Train-Serve/splitdata/test/', **data_gen_args)[0]

Found 22 images belonging to 4 classes.


In [13]:
img = test_set[0][1]

In [14]:
label = test_set[1][1]

In [15]:
img.shape

(224, 224, 3)

In [16]:
predict_batch = img.reshape((1,) + img.shape)

In [17]:
payload = {
          'instances': predict_batch.tolist()
           }

In [18]:
client = boto3.client('sagemaker-runtime')

In [21]:
endpoint_name = "tensorflow-inference-2020-05-21-22-47-12-683"                                       # Your endpoint name.
content_type = "application/json"                                        # The MIME type of the input data in the request body.
json_payload = json.dumps(payload)

In [22]:
response = client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType=content_type,
    Body=json_payload
    )


In [23]:
eval(response['Body'].read())


{'predictions': [[0.996722519, 0.000816869608, 2.01071157e-06, 0.00245864177]]}

In [24]:
label

array([1., 0., 0., 0.], dtype=float32)